In [56]:
import cv2
import mediapipe as mp
import numpy as np
import time

In [52]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

In [32]:
# Ordem dos pontos da face (P3, P5, P2, P6, P4, P1)
p_olho_esq = [385, 380, 387, 373, 362, 263]
p_olho_dir = [160, 144, 158, 153, 33, 133]
p_olhos = p_olho_esq + p_olho_dir

In [45]:
def calculo_ear(face, p_olho_dir, p_olho_esq):
    try:
        face = np.array([[coord.x, coord.y] for coord in face])
        face_esq = face[p_olho_esq, :]
        face_dir = face[p_olho_dir, :]

        # Calculo da distancia euclidiana entre os pontos
        ear_esq = np.linalg.norm(face_esq[0] - face_esq[1])+np.linalg.norm(face_esq[2] - face_esq[3])/(2*(np.linalg.norm(face_esq[4] - face_esq[5])))
        ear_dir = np.linalg.norm(face_esq[0] - face_esq[1])+np.linalg.norm(face_esq[2] - face_esq[3])/(2*(np.linalg.norm(face_esq[4] - face_esq[5])))
    except:
        ear_esq = 0
        ear_dir = 0
    media_ear = (ear_esq + ear_dir)/2
    return media_ear

In [73]:
p_boca = [82, 87, 13, 14, 312, 317, 78, 308]

In [76]:
def calculo_mar(face, p_boca):
    try:
        face = np.array([[coord.x, coord.y] for coord in face])
        face_boca = face[p_boca, :]
        mar = (np.linalg.norm(face_boca[0]-face_boca[1])+np.linalg.norm(face_boca[2]-face_boca[3])+np.linalg.norm(face_boca[4]-face_boca[5]))/(2*(np.linalg.norm(face_boca[6]-face_boca[7])))
    except:
        mar = 0
    return mar

In [100]:
ear_limiar = 0.15
mar_limiar = 0.20
fechado = 0
contagem_piscadas = 0
t_piscadas = time.time()
c_tempo = 0
contagem_temporaria = 0
contagem_lista = []

cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        sucesso, frame = cap.read()
        cv2.imshow('Camera', frame)
        if not sucesso:
            print("Ignorando o Frame vazio da Webcam")
            continue
        comprimento, largura, _ = frame.shape

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        saida_facemesh = face_mesh.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        try:
            for face_landmarks in saida_facemesh.multi_face_landmarks:
                face = face_landmarks.landmark

                # Desenhando os pontos dos olhos
                for id_coord, coord_xyz in enumerate(face):
                    if id_coord in p_olhos:
                       coord_cv = mp_drawing._normalized_to_pixel_coordinates(coord_xyz.x,coord_xyz.y, largura, comprimento)
                       cv2.circle(frame, coord_cv, 2, (255,0,0), -1)

                # Calculando o EAR (Eye Aspect Ratio)
                ear = calculo_ear(face,p_olho_dir, p_olho_esq)
                cv2.rectangle(frame, (0,1),(235,150),(58,58,55),-1)
                cv2.putText(frame, f"EAR: {round(ear, 2)}", (1, 24),
                                cv2.FONT_HERSHEY_DUPLEX,
                                0.9, (255, 255, 255), 2)
                
               

                    
                
                # Desenhando os pontos da boca
                for id_coord, coord_xyz in enumerate(face):
                    if id_coord in p_boca:
                       coord_cv = mp_drawing._normalized_to_pixel_coordinates(coord_xyz.x,coord_xyz.y, largura, comprimento)
                       cv2.circle(frame, coord_cv, 2, (255,0,0), -1)

                # Calculando o MAR (Mouth Aspect Ratio)
                mar = calculo_mar(face,p_boca)
                cv2.putText(frame, f"MAR: {round(mar, 2)}", (1, 50),
                                cv2.FONT_HERSHEY_DUPLEX,
                                0.9, (255, 255, 255), 2)
                
                # Calculando o tempo em que os olhos estão fechados e a boca fechada também
                if ear < ear_limiar and mar < mar_limiar:
                    t_inicial = time.time() if fechado == 0 else t_inicial
                    contagem_piscadas += 1 if fechado == 0 else 0
                    fechado = 1
                if (fechado == 1 and ear >= ear_limiar) or (fechado == 1 and mar >= mar_limiar):
                    fechado = 0
                t_final =  time.time()
                tempo_decorrido = t_final - t_piscadas

                if tempo_decorrido >= (c_tempo+1):
                    c_tempo = tempo_decorrido
                    piscadas_ps = contagem_piscadas-contagem_temporaria
                    contagem_temporaria = contagem_piscadas
                    contagem_lista.append(piscadas_ps)
                    contagem_lista = contagem_lista if (len(contagem_lista)<=60) else contagem_lista[-60:]
                piscadas_pm = 15 if tempo_decorrido<=60 else sum(contagem_lista)
                
                tempo = (t_final - t_inicial) if fechado == 1 else 0
                cv2.putText(frame, f"Tempo: {round(tempo, 3)}", (1, 80),
                                cv2.FONT_HERSHEY_DUPLEX,
                                0.9, (255, 255, 255), 2)
                cv2.putText(frame, f"Piscadas: {contagem_piscadas}", (1, 110),
                                cv2.FONT_HERSHEY_DUPLEX,
                                0.9, (255, 255, 255), 2)
                cv2.putText(frame, f"Por minuto: {round(piscadas_pm, 2)}", (1, 140),
                                cv2.FONT_HERSHEY_DUPLEX,
                                0.9, (255, 255, 255), 2)
                if piscadas_pm < 15 or tempo > 3:
                    cv2.rectangle(frame, (30,400),(610,452),(0, 0, 255),-1)
                    cv2.putText(frame, "OLHOS FECHADOS POR MUITO TEMPO", (80, 435),
                                cv2.FONT_HERSHEY_DUPLEX,
                                0.85, (255, 255, 255), 1)
                        
        except:
            pass
                                      
        cv2.imshow('Camera', frame)
        if cv2.waitKey(10) & 0xFF == ord('c'):
            break
cap.release()
cv2.destroyAllWindows()